In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import pnn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [4]:
def gen_pattern_replace_and_matcher_for_loop_pnn(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          return self.pn(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim= -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = -1)


    def forward(self,x):
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)

      return cross_term
    
  
  return pattern,ReplacementClass(),_match

In [6]:
# 只修改哈达玛积
def workload_pnn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096):
  print(f"now gen workload of PNN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  pnn_loop = pnn.ProductNeuralNetworkModel([50 for i in range(num_field)],dim,l,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(num_field)],dim,l,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [6]:
def time_evaluation(origin, compiled, input, exec_func=None, exp_name: str = '', warmup_time: int = 5) -> None:
    torch.cuda.synchronize()
    s_t = time.time()
    exec_func(origin, input) if exec_func else origin(input)
    torch.cuda.synchronize()
    start_t1 = time.time() - s_t
    print(f"Normal firstly used time:{start_t1}s")

    torch.cuda.synchronize()
    s_t = time.time()
    exec_func(compiled, input) if exec_func else compiled(input)
    torch.cuda.synchronize()
    start_t2 = time.time() - s_t
    print(f"Compiled firstly used time:{start_t2}s")

    assert warmup_time >= 1
    for _ in range(warmup_time - 1):
        exec_func(compiled, input) if exec_func else compiled(input)

    t_1_total, t_2_total = 0., 0.
    for i in range(10):
        torch.cuda.synchronize()
        s_t = time.time()
        exec_func(origin, input) if exec_func else origin(input)
        torch.cuda.synchronize()
        t_1 = time.time() - s_t
        t_1_total += t_1

        torch.cuda.synchronize()
        s_t = time.time()
        exec_func(compiled, input) if exec_func else compiled(input)
        torch.cuda.synchronize()
        t_2 = time.time() - s_t
        t_2_total += t_2

        print(f"{i}:\n\tNormal used time:{t_1}s, \n\t"
              f"Compiled used time:{t_2}s")

    print(f"{exp_name}在编译前的首次运行时间为:{start_t1}秒")
    print(f"{exp_name}在编译后的首次运行时间为:{start_t2}秒")
    print(f"{exp_name}在后续运行过程中的加速比为:{t_1_total / t_2_total:.2f}")

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_pnn):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    model.eval()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    for i in range(10):
       soutput = compiled_model(t)
    torch.cuda.synchronize()
    start = time.time()
    for i in range(100):
        soutput = compiled_model(t)
    end = time.time()     
    torch.cuda.synchronize()   
    print(f"cal time : {(end - start)/100 * 1000}")
  print(f"now gen workload of pnn with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,batch = batch)
  run(ori)
  run(modify)

In [8]:
def gen_model(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_pnn,l = [1024,512,256]):
    ori, modify = workload_func(num_field,prefix,dim,l,batch = batch)
    t = torch.randint(low=0, high=30, size=(batch,num_field), dtype=torch.long).cuda()
    def compile_model(model):
        model.eval()
        model = model.cuda()
        return torch.compile(model)
    ori_compile = compile_model(ori)
    modify_compile = compile_model(modify)
    print("=======ori========")
    time_evaluation(ori,ori_compile,t)
    print("=======modify========")
    time_evaluation(modify,modify_compile,t)

In [10]:
gen_model(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50
=======ori========
Normal firstly used time:0.21283626556396484s
Compiled firstly used time:2.517148017883301s
0:
	Normal used time:0.010285615921020508s, 
	Compiled used time:0.0025017261505126953s
1:
	Normal used time:0.010934114456176758s, 
	Compiled used time:0.002635955810546875s
2:
	Normal used time:0.010417699813842773s, 
	Compiled used time:0.002560853958129883s
3:
	Normal used time:0.010410308837890625s, 
	Compiled used time:0.002565622329711914s
4:
	Normal used time:0.010352373123168945s, 
	Compiled used time:0.002553224563598633s
5:
	Normal used time:0.01035165786743164s, 
	Compiled used time:0.002577543258666992s
6:
	Normal used time:0.010428905487060547s, 
	Compiled used time:0.002577543258666992s
7:
	Normal used time:0.010429143905639648s, 
	Compiled used time:0.002574920654296875s
8:
	Normal used time:0.010489702224731445s, 
	Compiled used time:0.002553224563598633s
9:
	Normal used time:0.0103933811

In [9]:
gen_model(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50
=======ori========
Normal firstly used time:0.21320128440856934s


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


Compiled firstly used time:5.28911280632019s
0:
	Normal used time:0.01983785629272461s, 
	Compiled used time:0.0042994022369384766s
1:
	Normal used time:0.020218372344970703s, 
	Compiled used time:0.0041468143463134766s
2:
	Normal used time:0.01994776725769043s, 
	Compiled used time:0.004144430160522461s
3:
	Normal used time:0.019878149032592773s, 
	Compiled used time:0.004130840301513672s
4:
	Normal used time:0.019887447357177734s, 
	Compiled used time:0.004189729690551758s
5:
	Normal used time:0.020119905471801758s, 
	Compiled used time:0.004193305969238281s
6:
	Normal used time:0.020013809204101562s, 
	Compiled used time:0.004140138626098633s
7:
	Normal used time:0.01992654800415039s, 
	Compiled used time:0.004147529602050781s
8:
	Normal used time:0.02013540267944336s, 
	Compiled used time:0.004125356674194336s
9:
	Normal used time:0.019928693771362305s, 
	Compiled used time:0.004195213317871094s
在编译前的首次运行时间为:0.21320128440856934秒
在编译后的首次运行时间为:5.28911280632019秒
在后续运行过程中的加速比为:4.79
===

In [10]:
gen_model(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50
=======ori========
Normal firstly used time:0.04418540000915527s
Compiled firstly used time:3.0754992961883545s
0:
	Normal used time:0.04074835777282715s, 
	Compiled used time:0.008039236068725586s
1:
	Normal used time:0.0395200252532959s, 
	Compiled used time:0.008051395416259766s
2:
	Normal used time:0.039321184158325195s, 
	Compiled used time:0.007986068725585938s
3:
	Normal used time:0.039220333099365234s, 
	Compiled used time:0.007913351058959961s
4:
	Normal used time:0.0390317440032959s, 
	Compiled used time:0.007961034774780273s
5:
	Normal used time:0.03935122489929199s, 
	Compiled used time:0.007879972457885742s
6:
	Normal used time:0.038521528244018555s, 
	Compiled used time:0.007876157760620117s
7:
	Normal used time:0.03899025917053223s, 
	Compiled used time:0.008031845092773438s
8:
	Normal used time:0.03951621055603027s, 
	Compiled used time:0.007916450500488281s
9:
	Normal used time:0.03926324844360351

In [11]:
gen_model(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145
=======ori========
Normal firstly used time:0.024608373641967773s
Compiled firstly used time:3.1552207469940186s
0:
	Normal used time:0.025131702423095703s, 
	Compiled used time:0.005460500717163086s
1:
	Normal used time:0.02486276626586914s, 
	Compiled used time:0.0051877498626708984s
2:
	Normal used time:0.02460002899169922s, 
	Compiled used time:0.005208730697631836s
3:
	Normal used time:0.024443864822387695s, 
	Compiled used time:0.005159616470336914s
4:
	Normal used time:0.024692296981811523s, 
	Compiled used time:0.005233049392700195s
5:
	Normal used time:0.024413585662841797s, 
	Compiled used time:0.005139350891113281s
6:
	Normal used time:0.02494192123413086s, 
	Compiled used time:0.0052144527435302734s
7:
	Normal used time:0.024571895599365234s, 
	Compiled used time:0.005141496658325195s
8:
	Normal used time:0.02462029457092285s, 
	Compiled used time:0.005234956741333008s
9:
	Normal used time:0.024389505

In [12]:
gen_model(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145
=======ori========
Normal firstly used time:0.04643559455871582s
Compiled firstly used time:2.9662179946899414s
0:
	Normal used time:0.047943115234375s, 
	Compiled used time:0.008634328842163086s
1:
	Normal used time:0.04665112495422363s, 
	Compiled used time:0.008735179901123047s
2:
	Normal used time:0.04644942283630371s, 
	Compiled used time:0.008817434310913086s
3:
	Normal used time:0.0471343994140625s, 
	Compiled used time:0.008624553680419922s
4:
	Normal used time:0.04710674285888672s, 
	Compiled used time:0.008587360382080078s
5:
	Normal used time:0.04676628112792969s, 
	Compiled used time:0.00863504409790039s
6:
	Normal used time:0.046413421630859375s, 
	Compiled used time:0.008721351623535156s
7:
	Normal used time:0.0468597412109375s, 
	Compiled used time:0.008590459823608398s
8:
	Normal used time:0.04699587821960449s, 
	Compiled used time:0.00859212875366211s
9:
	Normal used time:0.047033071517944336s, 


In [13]:
gen_model(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145
=======ori========
Normal firstly used time:0.09068179130554199s
Compiled firstly used time:0.8018302917480469s
0:
	Normal used time:0.09417009353637695s, 
	Compiled used time:0.016812801361083984s
1:
	Normal used time:0.09233951568603516s, 
	Compiled used time:0.016607046127319336s
2:
	Normal used time:0.0921623706817627s, 
	Compiled used time:0.016824722290039062s
3:
	Normal used time:0.09224867820739746s, 
	Compiled used time:0.016585111618041992s
4:
	Normal used time:0.0922236442565918s, 
	Compiled used time:0.01689910888671875s
5:
	Normal used time:0.09170818328857422s, 
	Compiled used time:0.01667475700378418s
6:
	Normal used time:0.09232044219970703s, 
	Compiled used time:0.016649484634399414s
7:
	Normal used time:0.09183049201965332s, 
	Compiled used time:0.016612768173217773s
8:
	Normal used time:0.09260320663452148s, 
	Compiled used time:0.016682863235473633s
9:
	Normal used time:0.09187602996826172s, 


In [14]:
gen_model(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 10
=======ori========
Normal firstly used time:0.03583359718322754s
Compiled firstly used time:1.905519723892212s
0:
	Normal used time:0.03387284278869629s, 
	Compiled used time:0.0070149898529052734s
1:
	Normal used time:0.033463239669799805s, 
	Compiled used time:0.00692296028137207s
2:
	Normal used time:0.03330254554748535s, 
	Compiled used time:0.00690770149230957s
3:
	Normal used time:0.03315019607543945s, 
	Compiled used time:0.006885528564453125s
4:
	Normal used time:0.03325223922729492s, 
	Compiled used time:0.007052898406982422s
5:
	Normal used time:0.03332018852233887s, 
	Compiled used time:0.006853342056274414s
6:
	Normal used time:0.03311467170715332s, 
	Compiled used time:0.006840944290161133s
7:
	Normal used time:0.03314566612243652s, 
	Compiled used time:0.006834983825683594s
8:
	Normal used time:0.03309750556945801s, 
	Compiled used time:0.006851673126220703s
9:
	Normal used time:0.033094167709350586s

In [15]:
gen_model(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 20
=======ori========
Normal firstly used time:0.031653642654418945s
Compiled firstly used time:0.5261111259460449s
0:
	Normal used time:0.03318643569946289s, 
	Compiled used time:0.0068585872650146484s
1:
	Normal used time:0.032869577407836914s, 
	Compiled used time:0.006781101226806641s
2:
	Normal used time:0.03271198272705078s, 
	Compiled used time:0.006781578063964844s
3:
	Normal used time:0.03283953666687012s, 
	Compiled used time:0.0068590641021728516s
4:
	Normal used time:0.03276395797729492s, 
	Compiled used time:0.006791353225708008s
5:
	Normal used time:0.03296256065368652s, 
	Compiled used time:0.006790637969970703s
6:
	Normal used time:0.03277444839477539s, 
	Compiled used time:0.0067901611328125s
7:
	Normal used time:0.03283882141113281s, 
	Compiled used time:0.0067980289459228516s
8:
	Normal used time:0.03295111656188965s, 
	Compiled used time:0.006787538528442383s
9:
	Normal used time:0.032930374145507

In [16]:
gen_model(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 30
=======ori========
Normal firstly used time:0.033225297927856445s
Compiled firstly used time:0.5285346508026123s
0:
	Normal used time:0.03358173370361328s, 
	Compiled used time:0.007065773010253906s
1:
	Normal used time:0.032648563385009766s, 
	Compiled used time:0.006858110427856445s
2:
	Normal used time:0.03243613243103027s, 
	Compiled used time:0.006863117218017578s
3:
	Normal used time:0.032346487045288086s, 
	Compiled used time:0.006822347640991211s
4:
	Normal used time:0.03244328498840332s, 
	Compiled used time:0.00669097900390625s
5:
	Normal used time:0.03242826461791992s, 
	Compiled used time:0.006831169128417969s
6:
	Normal used time:0.03233766555786133s, 
	Compiled used time:0.006821870803833008s
7:
	Normal used time:0.03226828575134277s, 
	Compiled used time:0.006770610809326172s
8:
	Normal used time:0.03238940238952637s, 
	Compiled used time:0.0068013668060302734s
9:
	Normal used time:0.032536029815673

In [17]:
gen_model(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 40
=======ori========
Normal firstly used time:0.032065391540527344s
Compiled firstly used time:0.7893524169921875s
0:
	Normal used time:0.03340768814086914s, 
	Compiled used time:0.006822347640991211s
1:
	Normal used time:0.03287053108215332s, 
	Compiled used time:0.006796836853027344s
2:
	Normal used time:0.032277822494506836s, 
	Compiled used time:0.006801605224609375s
3:
	Normal used time:0.033229827880859375s, 
	Compiled used time:0.00682520866394043s
4:
	Normal used time:0.03297114372253418s, 
	Compiled used time:0.0068013668060302734s
5:
	Normal used time:0.03318595886230469s, 
	Compiled used time:0.006804943084716797s
6:
	Normal used time:0.03298687934875488s, 
	Compiled used time:0.0068018436431884766s
7:
	Normal used time:0.032965898513793945s, 
	Compiled used time:0.0068285465240478516s
8:
	Normal used time:0.03301572799682617s, 
	Compiled used time:0.006808042526245117s
9:
	Normal used time:0.032973289489

In [18]:
gen_model(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 50
=======ori========
Normal firstly used time:0.031682729721069336s
Compiled firstly used time:0.5360007286071777s
0:
	Normal used time:0.03349924087524414s, 
	Compiled used time:0.0070497989654541016s
1:
	Normal used time:0.032741546630859375s, 
	Compiled used time:0.006917476654052734s
2:
	Normal used time:0.03260660171508789s, 
	Compiled used time:0.006906270980834961s
3:
	Normal used time:0.03246951103210449s, 
	Compiled used time:0.006915569305419922s
4:
	Normal used time:0.03255176544189453s, 
	Compiled used time:0.00690150260925293s
5:
	Normal used time:0.03269386291503906s, 
	Compiled used time:0.0068683624267578125s
6:
	Normal used time:0.032486677169799805s, 
	Compiled used time:0.006867170333862305s
7:
	Normal used time:0.03247690200805664s, 
	Compiled used time:0.0068700313568115234s
8:
	Normal used time:0.03244161605834961s, 
	Compiled used time:0.006876707077026367s
9:
	Normal used time:0.0325119495391

In [19]:
gen_model(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 60
=======ori========
Normal firstly used time:0.040042877197265625s
Compiled firstly used time:0.5171034336090088s
0:
	Normal used time:0.033696651458740234s, 
	Compiled used time:0.0069980621337890625s
1:
	Normal used time:0.03313016891479492s, 
	Compiled used time:0.006948709487915039s
2:
	Normal used time:0.033052921295166016s, 
	Compiled used time:0.007135868072509766s
3:
	Normal used time:0.03313732147216797s, 
	Compiled used time:0.007174968719482422s
4:
	Normal used time:0.0334467887878418s, 
	Compiled used time:0.006926059722900391s
5:
	Normal used time:0.03328657150268555s, 
	Compiled used time:0.006919384002685547s
6:
	Normal used time:0.03311443328857422s, 
	Compiled used time:0.006917238235473633s
7:
	Normal used time:0.03310108184814453s, 
	Compiled used time:0.0070362091064453125s
8:
	Normal used time:0.033103227615356445s, 
	Compiled used time:0.006903648376464844s
9:
	Normal used time:0.0330917835235

In [20]:
gen_model(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 70
=======ori========
Normal firstly used time:0.03183388710021973s
Compiled firstly used time:0.8099243640899658s
0:
	Normal used time:0.03366732597351074s, 
	Compiled used time:0.007073640823364258s
1:
	Normal used time:0.03284621238708496s, 
	Compiled used time:0.0069124698638916016s
2:
	Normal used time:0.0326685905456543s, 
	Compiled used time:0.00691533088684082s
3:
	Normal used time:0.03256654739379883s, 
	Compiled used time:0.006914377212524414s
4:
	Normal used time:0.032575130462646484s, 
	Compiled used time:0.0069234371185302734s
5:
	Normal used time:0.03280830383300781s, 
	Compiled used time:0.006885051727294922s
6:
	Normal used time:0.032469749450683594s, 
	Compiled used time:0.0068743228912353516s
7:
	Normal used time:0.03245043754577637s, 
	Compiled used time:0.006881237030029297s
8:
	Normal used time:0.032518625259399414s, 
	Compiled used time:0.0068438053131103516s
9:
	Normal used time:0.0324783325195

In [21]:
gen_model(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 80
=======ori========
Normal firstly used time:0.03188729286193848s
Compiled firstly used time:0.5166347026824951s
0:
	Normal used time:0.03330540657043457s, 
	Compiled used time:0.006908416748046875s
1:
	Normal used time:0.03296852111816406s, 
	Compiled used time:0.006825447082519531s
2:
	Normal used time:0.03281998634338379s, 
	Compiled used time:0.006829261779785156s
3:
	Normal used time:0.03291678428649902s, 
	Compiled used time:0.006861209869384766s
4:
	Normal used time:0.03282880783081055s, 
	Compiled used time:0.006832122802734375s
5:
	Normal used time:0.03285837173461914s, 
	Compiled used time:0.006831645965576172s
6:
	Normal used time:0.03283381462097168s, 
	Compiled used time:0.006831645965576172s
7:
	Normal used time:0.032846689224243164s, 
	Compiled used time:0.006835460662841797s
8:
	Normal used time:0.033020973205566406s, 
	Compiled used time:0.006897926330566406s
9:
	Normal used time:0.0329995155334472

In [22]:
gen_model(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 90
=======ori========
Normal firstly used time:0.03163766860961914s
Compiled firstly used time:0.5206866264343262s
0:
	Normal used time:0.03379702568054199s, 
	Compiled used time:0.006936788558959961s
1:
	Normal used time:0.03321385383605957s, 
	Compiled used time:0.006890535354614258s
2:
	Normal used time:0.0331110954284668s, 
	Compiled used time:0.006883859634399414s
3:
	Normal used time:0.03310894966125488s, 
	Compiled used time:0.006884574890136719s
4:
	Normal used time:0.033136606216430664s, 
	Compiled used time:0.006870746612548828s
5:
	Normal used time:0.03336811065673828s, 
	Compiled used time:0.00684666633605957s
6:
	Normal used time:0.033147573471069336s, 
	Compiled used time:0.006851911544799805s
7:
	Normal used time:0.03311729431152344s, 
	Compiled used time:0.0068666934967041016s
8:
	Normal used time:0.03312563896179199s, 
	Compiled used time:0.006852149963378906s
9:
	Normal used time:0.03311777114868164

In [ ]:
gen_model(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 99
=======ori========
Normal firstly used time:0.03159499168395996s
Compiled firstly used time:0.8287291526794434s
0:
	Normal used time:0.03313183784484863s, 
	Compiled used time:0.00696563720703125s
1:
	Normal used time:0.032776594161987305s, 
	Compiled used time:0.00683903694152832s
2:
	Normal used time:0.03267669677734375s, 
	Compiled used time:0.006838798522949219s
3:
	Normal used time:0.03271174430847168s, 
	Compiled used time:0.006838321685791016s
4:
	Normal used time:0.03262734413146973s, 
	Compiled used time:0.006844520568847656s
5:
	Normal used time:0.032950639724731445s, 
	Compiled used time:0.006840705871582031s
6:
	Normal used time:0.03269147872924805s, 
	Compiled used time:0.0068433284759521484s
7:
	Normal used time:0.03272509574890137s, 
	Compiled used time:0.006845951080322266s
8:
	Normal used time:0.03273200988769531s, 
	Compiled used time:0.00684666633605957s
9:
	Normal used time:0.03272199630737305s

: 

In [ ]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

In [19]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50
total:0.008240442276000976
total:0.008294439315795899


In [23]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50
total:0.021510486602783204
total:0.01708883762359619


In [47]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8135 / 8192 (99.3%)
Greatest absolute difference: 0.006009519100189209 at index (7302,) (up to 1e-05 allowed)
Greatest relative difference: 0.012253011693852506 at index (7302,) (up to 1e-05 allowed)
  _check_trace(


(76.9975148415079, 0.05092847441645147)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8131 / 8192 (99.3%)
Greatest absolute difference: 0.008351743221282959 at index (5262,) (up to 1e-05 allowed)
Greatest relative difference: 0.01642859186312254 at index (5262,) (up to 1e-05 allowed)
  _check_trace(


(54.037890531578846, 0.07083266857333502)


In [48]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1018 / 1024 (99.4%)
Greatest absolute difference: 0.005056828260421753 at index (659,) (up to 1e-05 allowed)
Greatest relative difference: 0.010316418858558864 at index (659,) (up to 1e-05 allowed)
  _check_trace(


(16.520793350375428, 0.009254436447729482)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1010 / 1024 (98.6%)
Greatest absolute difference: 0.004266917705535889 at index (487,) (up to 1e-05 allowed)
Greatest relative difference: 0.008657715470682174 at index (487,) (up to 1e-05 allowed)
  _check_trace(


(11.275855093586202, 0.002345704090666248)


In [49]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2019 / 2048 (98.6%)
Greatest absolute difference: 0.0037710070610046387 at index (909,) (up to 1e-05 allowed)
Greatest relative difference: 0.007477625451181508 at index (909,) (up to 1e-05 allowed)
  _check_trace(


(33.30982068120217, 0.013581332795280293)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2030 / 2048 (99.1%)
Greatest absolute difference: 0.004865020513534546 at index (781,) (up to 1e-05 allowed)
Greatest relative difference: 0.009892901791604071 at index (781,) (up to 1e-05 allowed)
  _check_trace(


(22.449288835330886, 0.009448788844236014)


In [50]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4056 / 4096 (99.0%)
Greatest absolute difference: 0.005174219608306885 at index (3620,) (up to 1e-05 allowed)
Greatest relative difference: 0.010165012096047064 at index (3620,) (up to 1e-05 allowed)
  _check_trace(


(66.57140685100944, 0.03512955941766663)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4068 / 4096 (99.3%)
Greatest absolute difference: 0.005175262689590454 at index (118,) (up to 1e-05 allowed)
Greatest relative difference: 0.010571626876781402 at index (118,) (up to 1e-05 allowed)
  _check_trace(


(45.045933820763416, 0.05087950718742977)


In [51]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8100 / 8192 (98.9%)
Greatest absolute difference: 0.004488587379455566 at index (112,) (up to 1e-05 allowed)
Greatest relative difference: 0.008767540378117496 at index (112,) (up to 1e-05 allowed)
  _check_trace(


(135.03402289565727, 0.2644119402003142)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8117 / 8192 (99.1%)
Greatest absolute difference: 0.0057509541511535645 at index (6767,) (up to 1e-05 allowed)
Greatest relative difference: 0.011284772884077306 at index (6767,) (up to 1e-05 allowed)
  _check_trace(


(90.87044307163784, 0.13257611473922482)


In [52]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1013 / 1024 (98.9%)
Greatest absolute difference: 0.0039010047912597656 at index (595,) (up to 1e-05 allowed)
Greatest relative difference: 0.0077968613130077425 at index (595,) (up to 1e-05 allowed)
  _check_trace(


(23.419263956498128, 0.054436304371648384)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1018 / 1024 (99.4%)
Greatest absolute difference: 0.004433572292327881 at index (183,) (up to 1e-05 allowed)
Greatest relative difference: 0.008759313018701639 at index (183,) (up to 1e-05 allowed)
  _check_trace(


(7.28982070514134, 0.006088313175655225)


In [24]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145
total:0.024238436222076415
total:0.00827463150024414


In [54]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4059 / 4096 (99.1%)
Greatest absolute difference: 0.004714250564575195 at index (1189,) (up to 1e-05 allowed)
Greatest relative difference: 0.009462165067835692 at index (1189,) (up to 1e-05 allowed)
  _check_trace(


(93.51413929219149, 0.018515229527542768)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4068 / 4096 (99.3%)
Greatest absolute difference: 0.005229681730270386 at index (1263,) (up to 1e-05 allowed)
Greatest relative difference: 0.010669499073648966 at index (1263,) (up to 1e-05 allowed)
  _check_trace(


(27.628443095148825, 0.025193376746832405)


In [56]:
torch.cuda.empty_cache()

In [57]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8099 / 8192 (98.9%)
Greatest absolute difference: 0.005339503288269043 at index (968,) (up to 1e-05 allowed)
Greatest relative difference: 0.010614523087612649 at index (968,) (up to 1e-05 allowed)
  _check_trace(


(181.41492851413025, 0.6386455073195321)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8116 / 8192 (99.1%)
Greatest absolute difference: 0.005984246730804443 at index (2406,) (up to 1e-05 allowed)
Greatest relative difference: 0.01206809840935109 at index (2406,) (up to 1e-05 allowed)
  _check_trace(


(53.12040628705706, 0.058460021273091044)


In [59]:
torch.cuda.empty_cache()

In [60]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1015 / 1024 (99.1%)
Greatest absolute difference: 0.004168868064880371 at index (605,) (up to 1e-05 allowed)
Greatest relative difference: 0.008437735868176754 at index (605,) (up to 1e-05 allowed)
  _check_trace(


(38.86586492888782, 0.09363474744694264)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1017 / 1024 (99.3%)
Greatest absolute difference: 0.00404590368270874 at index (463,) (up to 1e-05 allowed)
Greatest relative difference: 0.007893621992038581 at index (463,) (up to 1e-05 allowed)
  _check_trace(


(10.413320570575948, 0.02010468516393255)


In [61]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2022 / 2048 (98.7%)
Greatest absolute difference: 0.005228489637374878 at index (1380,) (up to 1e-05 allowed)
Greatest relative difference: 0.010564229029985203 at index (1380,) (up to 1e-05 allowed)
  _check_trace(


(78.12557345020528, 0.07990585846139096)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2028 / 2048 (99.0%)
Greatest absolute difference: 0.004784494638442993 at index (586,) (up to 1e-05 allowed)
Greatest relative difference: 0.00963675471610678 at index (586,) (up to 1e-05 allowed)
  _check_trace(


(20.925053985751404, 0.039567889043959845)


In [66]:
torch.cuda.empty_cache()

In [67]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4047 / 4096 (98.8%)
Greatest absolute difference: 0.004284709692001343 at index (1312,) (up to 1e-05 allowed)
Greatest relative difference: 0.00852385340236162 at index (1312,) (up to 1e-05 allowed)
  _check_trace(


(157.0282222202846, 0.37839874352461517)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4069 / 4096 (99.3%)
Greatest absolute difference: 0.0046863555908203125 at index (2120,) (up to 1e-05 allowed)
Greatest relative difference: 0.009548523084280799 at index (2120,) (up to 1e-05 allowed)
  _check_trace(


(41.74119964911013, 0.029043787107327274)


In [69]:
torch.cuda.empty_cache()

In [8]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


Traceback (most recent call last):
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph_module.py", line 304, in __call__
    return super(self.cls, obj).__call__(*args, **kwargs)  # type: ignore[misc]
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1522, in _slow_forward
    result = self.forward(*input, **kwargs)
  File "<eval_with_key>.0 from /home/yssun/pytorch-fm/torchfm/model/test_fx/exp/../../pnn.py:54 in forward", line 9, in forward
    getitem_1 = embedding_embedding[(slice(None, None, None), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.06 GiB. GPU 

In [8]:
gen_and_test(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 10, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4060 / 4096 (99.1%)
Greatest absolute difference: 0.005230695009231567 at index (2145,) (up to 1e-05 allowed)
Greatest relative difference: 0.010724332539180478 at index (2145,) (up to 1e-05 allowed)
  _check_trace(


(32.07204676647576, 0.030120181260126156)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4063 / 4096 (99.2%)
Greatest absolute difference: 0.005445539951324463 at index (1812,) (up to 1e-05 allowed)
Greatest relative difference: 0.01072899152928615 at index (1812,) (up to 1e-05 allowed)
  _check_trace(


(30.427802786535146, 0.023340050006547666)


In [9]:
gen_and_test(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 20, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 20


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4056 / 4096 (99.0%)
Greatest absolute difference: 0.004923462867736816 at index (1336,) (up to 1e-05 allowed)
Greatest relative difference: 0.010044738742488745 at index (1336,) (up to 1e-05 allowed)
  _check_trace(


(32.124231163336304, 0.025055042409513183)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4065 / 4096 (99.2%)
Greatest absolute difference: 0.0058058202266693115 at index (3346,) (up to 1e-05 allowed)
Greatest relative difference: 0.011864826090229465 at index (3346,) (up to 1e-05 allowed)
  _check_trace(


(28.450012498972367, 0.025394915324859133)


In [10]:
gen_and_test(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 30, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 30


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4067 / 4096 (99.3%)
Greatest absolute difference: 0.006021261215209961 at index (4093,) (up to 1e-05 allowed)
Greatest relative difference: 0.011615781410817257 at index (4093,) (up to 1e-05 allowed)
  _check_trace(


(32.339553716231364, 0.014590623970123931)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4064 / 4096 (99.2%)
Greatest absolute difference: 0.00461927056312561 at index (3872,) (up to 1e-05 allowed)
Greatest relative difference: 0.009299065473827453 at index (3872,) (up to 1e-05 allowed)
  _check_trace(


(26.361568275763062, 0.020832077633998053)


In [11]:
gen_and_test(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 40, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 40


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4055 / 4096 (99.0%)
Greatest absolute difference: 0.005202561616897583 at index (1416,) (up to 1e-05 allowed)
Greatest relative difference: 0.010661027817643286 at index (1416,) (up to 1e-05 allowed)
  _check_trace(


(32.473117361263355, 0.01664473747572786)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4064 / 4096 (99.2%)
Greatest absolute difference: 0.0045577287673950195 at index (1741,) (up to 1e-05 allowed)
Greatest relative difference: 0.008820348467552985 at index (1741,) (up to 1e-05 allowed)
  _check_trace(


(24.146536515683543, 0.008542814026956736)


In [12]:
gen_and_test(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4057 / 4096 (99.0%)
Greatest absolute difference: 0.005461394786834717 at index (2320,) (up to 1e-05 allowed)
Greatest relative difference: 0.010950032135327373 at index (2320,) (up to 1e-05 allowed)
  _check_trace(


(32.597200043347414, 0.017950223196886116)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4060 / 4096 (99.1%)
Greatest absolute difference: 0.005231320858001709 at index (1389,) (up to 1e-05 allowed)
Greatest relative difference: 0.01081128023261615 at index (1389,) (up to 1e-05 allowed)
  _check_trace(


(21.50522843185736, 0.007522454660511108)


In [13]:
gen_and_test(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 60, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 60


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4051 / 4096 (98.9%)
Greatest absolute difference: 0.004347681999206543 at index (2965,) (up to 1e-05 allowed)
Greatest relative difference: 0.008484169211615013 at index (2965,) (up to 1e-05 allowed)
  _check_trace(


(32.536956981736786, 0.00976909497735336)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4057 / 4096 (99.0%)
Greatest absolute difference: 0.006002962589263916 at index (2605,) (up to 1e-05 allowed)
Greatest relative difference: 0.011949970378103076 at index (2605,) (up to 1e-05 allowed)
  _check_trace(


(18.71425305580606, 0.005979468060654425)


In [14]:
gen_and_test(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 70, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 70


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4059 / 4096 (99.1%)
Greatest absolute difference: 0.005123436450958252 at index (236,) (up to 1e-05 allowed)
Greatest relative difference: 0.010119949605695532 at index (236,) (up to 1e-05 allowed)
  _check_trace(


(32.589922457325216, 0.017757480557320078)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4061 / 4096 (99.1%)
Greatest absolute difference: 0.005567222833633423 at index (1677,) (up to 1e-05 allowed)
Greatest relative difference: 0.011285174836691036 at index (1677,) (up to 1e-05 allowed)
  _check_trace(


(15.67338972675557, 0.014153214149986942)


In [15]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 80, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 80


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4062 / 4096 (99.2%)
Greatest absolute difference: 0.004798144102096558 at index (245,) (up to 1e-05 allowed)
Greatest relative difference: 0.009720646800157535 at index (245,) (up to 1e-05 allowed)
  _check_trace(


(32.66789050978057, 0.010930713658514748)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4059 / 4096 (99.1%)
Greatest absolute difference: 0.00475764274597168 at index (1210,) (up to 1e-05 allowed)
Greatest relative difference: 0.009295217120527595 at index (1210,) (up to 1e-05 allowed)
  _check_trace(


(12.399309372415347, 0.016523300131286542)


In [16]:
gen_and_test(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 90, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 90


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4058 / 4096 (99.1%)
Greatest absolute difference: 0.0059822797775268555 at index (3565,) (up to 1e-05 allowed)
Greatest relative difference: 0.012051220629457324 at index (3565,) (up to 1e-05 allowed)
  _check_trace(


(32.72729102932677, 0.012171156126203854)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4059 / 4096 (99.1%)
Greatest absolute difference: 0.004232347011566162 at index (3307,) (up to 1e-05 allowed)
Greatest relative difference: 0.008353731290957189 at index (3307,) (up to 1e-05 allowed)
  _check_trace(


(8.803326061793737, 0.016811522319492688)


In [17]:
gen_and_test(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 99, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 99


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4063 / 4096 (99.2%)
Greatest absolute difference: 0.005911886692047119 at index (2402,) (up to 1e-05 allowed)
Greatest relative difference: 0.011564031635207821 at index (2402,) (up to 1e-05 allowed)
  _check_trace(


(32.272871309397175, 0.01394551536030842)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4069 / 4096 (99.3%)
Greatest absolute difference: 0.004561185836791992 at index (925,) (up to 1e-05 allowed)
Greatest relative difference: 0.009412662481252075 at index (925,) (up to 1e-05 allowed)
  _check_trace(


(5.2761306130156225, 0.00794410174053358)


In [15]:
def gen_and_test_profile(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_pnn):
  def run(model):
    t = torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long).cuda()
    model.eval()
    traced_model = torch.jit.trace(model.cuda(), t)
    traced_model.eval()
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []

    with profile(activities=[torch.profiler.ProfilerActivity.CUDA,ProfilerActivity.CPU], record_shapes=True,
        profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    with_flops=True,  # 记录FLOPs
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log_PNN_gpu')                 
                   
                   
                   ) as prof:
        with record_function("model_inference"):
          for i in range(6):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              torch.cuda.synchronize()
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,batch = batch)
  run(ori)
  run(modify)

In [16]:
gen_and_test_profile(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145
(22.763550281524658, 0.01074472422146755)
(6.942689418792725, 0.014125202341119802)


In [19]:
# ort

In [46]:
ori, modify = workload_pnn(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


In [47]:
torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(4096,34 * 5), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  '/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn_4096_ori.onnx',        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


In [48]:
torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(4096,34 * 5), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  '/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn_4096_modify.onnx',        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [ ]:
ori, modify = workload_pnn(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

In [7]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn_rate/pnn_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn_rate/pnn_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_pnn(num_field = num_field,prefix = prefix, batch = batch, dim = dim)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [8]:
dims= [32]
batches = [1024,2048,4096]
# num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]

In [9]:
for dim in dims:
  for batch in batches:
    for prefix in [10,20,30,40,50,60,70,80,90,99]:
      genWorkload(num_field=100,prefix=prefix,batch=batch,dim=dim)

now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 20


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 30


...

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 90


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 99


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


In [8]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34,29),(22 ,10 )]

In [9]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10
now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
